In [ ]:
'''Considere um dataset em formato CSV com informações estatísticas sobre as eleições
brasileiras do ano de 2016. (fonte http://www.tse.jus.br)
• O arquivo (BR_eleitorado_2016_municipio.csv) contém dados segmentados por município,
estado da federação, faixas etárias e gêneros. O arquivo foi gerado com encoding utf-8.
• O arquivo está sem um critério de ordenação
Pede-se:
• Construa uma função que efetue a leitura do arquivo, criando uma lista de objetos que
representem sua estrutura de dados do arquivo (classe Election2016)
• Gere um arquivo tipo CSV codificado em utf-8 com o nome pleito_2016_classificado.csv com
todas as informações lidas do arquivo anterior, classificadas pelas colunas uf e
cod_municipio_tse consecutivamente - FEITO

Construa uma função de nome group_female_by_uf() que deverá agrupar, sumarizar e retornar
uma lista de tuplas com a quantidade de eleitoras por estado (uf, qtd), utilizando o recurso list
comprehension (dica: utilize a função groupby() para agrupar e sum() para sumarizar)
• A partir da lista gerada pela função group_female_by_uf() gere um arquivo JSON com nome
pleito_2016_stat.json com as seguintes informações estatísticas:
• Média aritmética
• Média ponderada
• Mediana
• Variância
• Desvio Padrão
Feito

• Crie uma função chamada save_election_by_uf(). Esta função deverá gravar em arquivos CSV,
as mesmas informações do arquivo principal (BR_eleitorado_2016_municipio.csv) com a
mesma estrutura, porém separando um arquivo por unidade federativa (UF). Cada arquivo
deverá ter o nome da uf. Os arquivos deverão ser gravados por threads separadas favorecendo
dessa forma o desempenho do programa no momento da gravação.

In [54]:
#Imports 
import csv
from datetime import datetime
from operator import itemgetter, attrgetter
from itertools import groupby
import statistics
import json
import threading

In [55]:
# Construa uma função que efetue a leitura do arquivo, criando uma lista de objetos que
# representem sua estrutura de dados do arquivo (classe Election2016)

class Election2016(list):
    
    def __init__(self, table):
        self.cod_municipio_tse = table['cod_municipio_tse']
        self.uf= table['uf'] 
        self.nome_municipio = table['nome_municipio'] 
        self.total_eleitores=table['total_eleitores']
        self.f_16 = table['f_16'] 
        self.f_17 = table['f_17'] 
        self.f_18_20 = table['f_18_20'] 
        self.f_21_24 = table['f_21_24'] 
        self.f_25_34 = table['f_25_34'] 
        self.f_35_44 = table['f_35_44'] 
        self.f_45_59 = table['f_45_59'] 
        self.f_60_69 = table['f_60_69']
        self.f_70_79 = table['f_70_79']
        self.f_sup_79 = table['f_sup_79']
        self.gen_feminino = table['gen_feminino']
        self.gen_masculino = table['gen_masculino']
        self.gen_nao_informado = table['gen_nao_informado']
    
    def __str__(self):
        return str(vars(self))

In [56]:
def group_female_by_uf(eleitorado):
    listaEleitorado = list(map(vars, eleitorado)) #converte o dictionary pra list
    ufGenfemininoTuple = []
    for elei in listaEleitorado: #cria tupla uf, gen_feminino
        node = []
        node.append(elei['uf'])
        node.append(int(elei['gen_feminino']))
        ufGenfemininoTuple.append(node)
    
    somaAgrupado = list(somatorio(ufGenfemininoTuple)) #função somatorio calcula a soma de cada grupo
 
    createJSONStatistics(somaAgrupado)  #cria o arquivo JSON com o resultado das somas de cada grupo

In [63]:
#função que soma os valores de cada grupo de UF
def somatorio(l):
    it = groupby(l, itemgetter(0))
    for key, group in it:
            yield key, sum(item[1] for item in group) 

In [57]:
#funcao que gera o arquivo JSON com as estatisticas
#recebe como parametro a lista de tuplas com os grupos ja sumarizados
def createJSONStatistics(somaAgrupado):
    
    sumGenFem = []
    for item in somaAgrupado:
         sumGenFem.append(item[1])
            
    #Gera as estatisticas usando statistics         
    statMean = statistics.mean(sumGenFem)
    statMedian = statistics.median(sumGenFem)
    statVar = statistics.variance(sumGenFem)
    statStdev = statistics.stdev(sumGenFem)
    
    bodyJson = {
        "Media aritmetica": statMean,
        "Media ponderada" : 0,
        "Mediana" : statMedian,
        "Variancia" : statVar,
        "Desvio Padrao" : statStdev
    }
    
     #Salvando estatisticas no arquivo JSON
    with open('pleito_2016_stat.json', 'w') as outfile:
        json.dump(bodyJson, outfile,indent=4,)

In [59]:
# Construa uma função que efetue a leitura do arquivo,
def readCsvFile(fileName, outputFile):
    with open(fileName, 'r', encoding='utf-8') as f:
        # read customers into list if Customers
        reader = csv.DictReader(f, delimiter=';')
        eleitorado = [Election2016(c) for c in reader]
        
        # sort eleitorado by UF and cod_municipio_tse
        sorted_eleitorado_by_uf = sorted(eleitorado, key=attrgetter('uf', 'cod_municipio_tse'))

        # save CVS file
        header = ['cod_municipio_tse', 'uf','nome_municipio','total_eleitores','f_16','f_17','f_18_20','f_21_24','f_25_34','f_35_44','f_45_59','f_60_69','f_70_79','f_sup_79','gen_feminino','gen_masculino','gen_nao_informado']
        save_csv(outputFile, header, sorted_eleitorado_by_uf)

        group_female_by_uf(sorted_eleitorado_by_uf)
        
        save_election_by_uf(sorted_eleitorado_by_uf)

In [60]:
#Funcao pra salvar arquivo CSV
def save_csv(file, header, content):
    with open(file, 'w', newline='') as f:
        writer = csv.DictWriter(f, delimiter=';', fieldnames=header)
        writer.writeheader()
        try:
            writer.writerows(list(map(vars, content)))
        except:
            writer.writerows(content)

In [61]:
#funcao que cria as threads pra salvar os arquivos por estado
def save_election_by_uf(dadosOrdenados: Election2016):
    listaOrdenada = list(map(vars, dadosOrdenados))
    uf = ''
    threadList = []
    header = ['cod_municipio_tse', 'uf','nome_municipio','total_eleitores','f_16','f_17','f_18_20','f_21_24','f_25_34','f_35_44','f_45_59','f_60_69','f_70_79','f_sup_79','gen_feminino','gen_masculino','gen_nao_informado']
    
    
    for dados in listaOrdenada:
        uf = dados['uf']
        threads = []
        fileName = uf + '_eleitorado_2016_municipio.csv'
        dadosEleitoradoPorUf = []
        eleitoradoPorUf = (x for x in listaOrdenada if x['uf']==uf)
        
        for eleitorado in eleitoradoPorUf:
            dadosEleitoradoPorUf.append(eleitorado)
        #save_csv(fileName, header, dadosEleitoradoPorUf)   
        t = threading.Thread(target=save_csv, args=(fileName,header, dadosEleitoradoPorUf ))
        threads.append(t)
        t.start()

In [64]:
readCsvFile('BR_eleitorado_2016_municipio.csv', 'pleito_2016_classificado.csv')